In [1]:
!pip install kloppy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.0/139.0 kB 593.3 kB/s eta 0:00:00


In [2]:
from kloppy import datasets, wyscout
import json
import pandas as pd
import numpy as np

In [3]:
manu_westham = datasets.load('wyscout', match_id = 2499724)

In [4]:
teams_json = '../input/football-event-data/teams.json'
with open(teams_json) as f:
    data = json.load(f)
    teams = pd.json_normalize(data, sep = "_")
teams_map = dict()
for i, team in teams.iterrows():
    teams_map[team['wyId']] = team['name']
teams_map

{1613: 'Newcastle United',
 692: 'Celta de Vigo',
 691: 'Espanyol',
 696: 'Deportivo Alav\\u00e9s',
 695: 'Levante',
 3795: 'Troyes',
 698: 'Getafe',
 2454: "Borussia M'gladbach",
 1673: 'Huddersfield Town',
 678: 'Athletic Club',
 679: 'Atl\\u00e9tico Madrid',
 3766: 'Olympique Lyonnais',
 3767: 'PSG',
 674: 'Valencia',
 675: 'Real Madrid',
 676: 'Barcelona',
 714: 'Las Palmas',
 712: 'Legan\\u00e9s',
 3204: 'SPAL',
 10531: 'Swansea City',
 3771: 'Olympique Marseille',
 3770: 'Nantes',
 3775: 'Nice',
 3774: 'Rennes',
 3779: 'Strasbourg',
 701: 'Eibar',
 1659: 'AFC Bournemouth',
 1651: 'Brighton & Hove Albion',
 2443: 'Werder Bremen',
 2446: 'Bayer Leverkusen',
 2447: 'Borussia Dortmund',
 2444: 'Bayern M\\u00fcnchen',
 2445: 'Stuttgart',
 2449: 'Schalke 04',
 3157: 'Milan',
 3799: 'Angers',
 3159: 'Juventus',
 3158: 'Roma',
 3315: 'Sassuolo',
 1646: 'Burnley',
 3772: 'Bordeaux',
 2455: 'Hannover 96',
 3804: 'Dijon',
 2457: 'Hertha BSC',
 2451: 'Wolfsburg',
 2450: 'Hamburger SV',
 2453

In [5]:
json_path = '../input/wyscoutdata/matches_England.json'
with open(json_path) as f:
    data = json.load(f)
    matches = pd.json_normalize(data, sep = "_")
    
players_json = "../input/football-event-data/players.json"
with open(players_json) as f:
    data = json.load(f)
    players = pd.json_normalize(data, sep = "_")

In [6]:
import os
os.listdir('/kaggle/input/wyscoutdata')

['matches_European_Championship.json',
 'matches_France.json',
 'matches_Italy.json',
 'matches_Spain.json',
 'matches_World_Cup.json',
 'matches_England.json',
 'matches_Germany.json']

In [7]:
players_map = dict()
for i, player in players.iterrows():
    players_map[player['wyId']] = player['firstName'] + ' ' + player['middleName'] + ' ' + player['lastName']
players_map

{32777: 'Harun  Tekin',
 393228: 'Malang  Sarr',
 393230: 'Over  Mandanda',
 32793: "Alfred John Momar  N'Diaye",
 393247: 'Ibrahima  Konat\\u00e9',
 33: 'Jasper  Cillessen',
 36: 'Toby  Alderweireld',
 48: 'Jan  Vertonghen',
 229427: 'Alexander  Djiku',
 54: 'Christian   Dannemann Eriksen',
 65596: 'Marcel  Sabitzer',
 66: 'Kolbeinn  Sig\\u00fe\\u00f3rsson',
 393284: 'Charles  Traor\\u00e9',
 77: 'Niklas  Moisander',
 91: 'Charlison  Benschop',
 93: 'Johann   Berg Gu\\u00f0munds\\u00adson',
 99: 'Przemys\\u0142aw  Tyto\\u0144',
 102: 'Marcelo Ant\\u00f4nio  Guedes Filho',
 110: 'Jetro  Willems',
 131184: 'Ali Adnan Kadhim  Al Tameemi',
 114: 'Kevin  Strootman',
 118: 'Memphis  Depay',
 122: 'Dries  Mertens',
 123: 'Ola  Toivonen',
 130: 'Stefan  de Vrij',
 142: 'Karim  El Ahmadi Al Aroos',
 151: 'John  Guidetti',
 32923: 'Volkan  Babacan',
 159: 'Jeffrey  Gouweleeuw',
 174: 'Filip  \\u0110uri\\u010di\\u0107',
 32959: 'Peter  Pekar\\u00edk',
 192: 'Jago\\u0161  Vukovi\\u0107',
 32968: 

In [8]:
matches['gameweek']

0      38
1      38
2      38
3      38
4      38
       ..
375     1
376     1
377     1
378     1
379     1
Name: gameweek, Length: 380, dtype: int64

In [9]:
matches.label.iloc[0].split(' - ')[1].split(', ')

['AFC Bournemouth', '1']

In [10]:
def split_label(string):
    home, away_home_score, away_score = string.split(' - ')
    away, home_score = away_home_score.split(', ')
    return home, away, home_score, away_score

In [11]:
matches['home_team'],matches['away_team'], matches['home_score'], matches['away_score'] = zip(*matches['label'].map(split_label))

In [12]:
scores = matches[['home_team', 'away_team', 'home_score', 'away_score', 'gameweek']]

In [13]:
teams = matches['home_team'].unique()
teams

array(['Burnley', 'Crystal Palace', 'Huddersfield Town', 'Liverpool',
       'Manchester United', 'Newcastle United', 'Southampton',
       'Swansea City', 'Tottenham Hotspur', 'West Ham United',
       'Manchester City', 'Leicester City', 'Chelsea', 'Arsenal',
       'Everton', 'AFC Bournemouth', 'Watford', 'West Bromwich Albion',
       'Stoke City', 'Brighton & Hove Albion'], dtype=object)

In [14]:
scores['home_score'] = scores['home_score'].astype(np.int)
scores['away_score'] = scores['away_score'].astype(np.int)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython 

In [15]:
table = {}
for team in teams:
    table[team] = {'goals': 0, 'goals_against': 0, 'points': 0, 'wins': 0, 'draws': 0, 'losses': 0}
for index, game in scores.iterrows():
    home_team = game['home_team']
    away_team = game['away_team']
    away_score = game['away_score']
    home_score = game['home_score']
    #update scores
    table[home_team]['goals'] += home_score
    table[home_team]['goals_against'] += away_score
    table[away_team]['goals'] += away_score
    table[away_team]['goals_against'] += home_score
    #update wins
    if home_score > away_score:
        table[home_team]['wins'] += 1
        table[away_team]['losses'] += 1 
    elif home_score == away_score:
        table[home_team]['draws'] += 1
        table[away_team]['draws'] += 1
    else:
        table[home_team]['losses'] += 1
        table[away_team]['wins'] += 1

In [16]:
table = pd.DataFrame().from_dict(table, orient='index')

In [17]:
table['points'] = table['wins']*3 + table['draws']

In [18]:
table.sort_values(by = 'points')

,goals,goals_against,points,wins,draws,losses
West Bromwich Albion,31,56,31,6,13,19
Stoke City,35,68,33,7,12,19
Swansea City,28,56,33,8,9,21
Southampton,37,56,36,7,15,16
Huddersfield Town,28,58,37,9,10,19
Brighton & Hove Albion,34,54,40,9,13,16
Watford,44,64,41,11,8,19
West Ham United,48,68,42,10,12,16
AFC Bournemouth,45,61,44,11,11,16
Crystal Palace,45,55,44,11,11,16


In [19]:
table.to_csv("epl_table.csv")

In [20]:
df = matches
df = df.drop(df.var()[df.var() == 0].index.values, axis=1)
for col in df.columns:
    print(col)

status
gameweek
dateutc
winner
venue
wyId
label
date
referees
duration
teamsData_1646_coachId
teamsData_1646_side
teamsData_1646_score
teamsData_1646_formation_bench
teamsData_1646_formation_lineup
teamsData_1646_formation_substitutions
teamsData_1646_scoreHT
teamsData_1659_coachId
teamsData_1659_side
teamsData_1659_score
teamsData_1659_formation_bench
teamsData_1659_formation_lineup
teamsData_1659_formation_substitutions
teamsData_1659_scoreHT
teamsData_1628_coachId
teamsData_1628_side
teamsData_1628_score
teamsData_1628_formation_bench
teamsData_1628_formation_lineup
teamsData_1628_formation_substitutions
teamsData_1628_scoreHT
teamsData_1627_coachId
teamsData_1627_side
teamsData_1627_score
teamsData_1627_formation_bench
teamsData_1627_formation_lineup
teamsData_1627_formation_substitutions
teamsData_1627_scoreHT
teamsData_1609_coachId
teamsData_1609_side
teamsData_1609_score
teamsData_1609_formation_bench
teamsData_1609_formation_lineup
teamsData_1609_formation_substitutions
teamsDa

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  


In [21]:
week1 = matches[matches.gameweek == 1][['wyId', 'away_team', 'home_team', 'label']]
week1

,wyId,away_team,home_team,label
370,2499724,West Ham United,Manchester United,"Manchester United - West Ham United, 4 - 0"
371,2499725,Tottenham Hotspur,Newcastle United,"Newcastle United - Tottenham Hotspur, 0 - 2"
372,2499720,Manchester City,Brighton & Hove Albion,"Brighton & Hove Albion - Manchester City, 0 - 2"
373,2499721,Burnley,Chelsea,"Chelsea - Burnley, 2 - 3"
374,2499722,Huddersfield Town,Crystal Palace,"Crystal Palace - Huddersfield Town, 0 - 3"
375,2499723,Stoke City,Everton,"Everton - Stoke City, 1 - 0"
376,2499726,Swansea City,Southampton,"Southampton - Swansea City, 0 - 0"
377,2499728,AFC Bournemouth,West Bromwich Albion,"West Bromwich Albion - AFC Bournemouth, 1 - 0"
378,2499727,Liverpool,Watford,"Watford - Liverpool, 3 - 3"
379,2499719,Leicester City,Arsenal,"Arsenal - Leicester City, 4 - 3"


In [22]:
def map_player(x):
    try:
        return players_map[int(x)]
    except:
        return "NA"

In [23]:
for col in df.columns:
    print(col)

status
gameweek
dateutc
winner
venue
wyId
label
date
referees
duration
teamsData_1646_coachId
teamsData_1646_side
teamsData_1646_score
teamsData_1646_formation_bench
teamsData_1646_formation_lineup
teamsData_1646_formation_substitutions
teamsData_1646_scoreHT
teamsData_1659_coachId
teamsData_1659_side
teamsData_1659_score
teamsData_1659_formation_bench
teamsData_1659_formation_lineup
teamsData_1659_formation_substitutions
teamsData_1659_scoreHT
teamsData_1628_coachId
teamsData_1628_side
teamsData_1628_score
teamsData_1628_formation_bench
teamsData_1628_formation_lineup
teamsData_1628_formation_substitutions
teamsData_1628_scoreHT
teamsData_1627_coachId
teamsData_1627_side
teamsData_1627_score
teamsData_1627_formation_bench
teamsData_1627_formation_lineup
teamsData_1627_formation_substitutions
teamsData_1627_scoreHT
teamsData_1609_coachId
teamsData_1609_side
teamsData_1609_score
teamsData_1609_formation_bench
teamsData_1609_formation_lineup
teamsData_1609_formation_substitutions
teamsDa

In [24]:
games = []
for index, game in week1.iterrows():
    df = datasets.load('wyscout', match_id = game.wyId).to_pandas()
    teams = df.team_id.unique()
    df['team'] = df['team_id'].apply(lambda x: teams_map[int(x)])
    df['player'] = df['player_id'].apply(map_player)
    df['opponent'] = np.where(df['team_id'] == teams[0], teams_map[int(teams[1])], teams_map[int(teams[0])])
    games.append(df)

In [25]:
games[0][['team', 'opponent']]

,team,opponent
0,Manchester United,West Ham United
1,Manchester United,West Ham United
2,Manchester United,West Ham United
3,Manchester United,West Ham United
4,Manchester United,West Ham United
...,...,...
1641,West Ham United,Manchester United
1642,West Ham United,Manchester United
1643,Manchester United,West Ham United
1644,West Ham United,Manchester United


In [26]:
gameweek1 = pd.concat(games)

In [27]:
eventdf = gameweek1.groupby(['team', 'event_type']).size().reset_index()
eventdf[eventdf.event_type == 'PASS'].sort_values(by=0, ascending = False)

,team,event_type,0
73,Manchester City,PASS,795
119,Tottenham Hotspur,PASS,738
9,Arsenal,PASS,657
3,AFC Bournemouth,PASS,639
95,Southampton,PASS,562
30,Chelsea,PASS,552
44,Everton,PASS,539
80,Manchester United,PASS,511
66,Liverpool,PASS,499
125,Watford,PASS,427


In [28]:
eventdf.event_type.unique()

array(['CARD', 'FOUL_COMMITTED', 'GENERIC:generic', 'PASS', 'RECOVERY',
       'SHOT', 'TAKE_ON', 'BALL_OUT'], dtype=object)

In [29]:
eventdf[eventdf.groupby('event_type')[0].transform(max) == eventdf[0]]

,team,event_type,0
11,Arsenal,SHOT,27
28,Chelsea,CARD,6
43,Everton,GENERIC:generic,5
56,Leicester City,BALL_OUT,84
65,Liverpool,GENERIC:generic,5
67,Liverpool,RECOVERY,81
73,Manchester City,PASS,795
78,Manchester United,FOUL_COMMITTED,17
99,Stoke City,BALL_OUT,84
128,Watford,TAKE_ON,266


In [30]:
eventdf[eventdf[0] == eventdf.groupby('event_type')[0].transform('max')]

,team,event_type,0
11,Arsenal,SHOT,27
28,Chelsea,CARD,6
43,Everton,GENERIC:generic,5
56,Leicester City,BALL_OUT,84
65,Liverpool,GENERIC:generic,5
67,Liverpool,RECOVERY,81
73,Manchester City,PASS,795
78,Manchester United,FOUL_COMMITTED,17
99,Stoke City,BALL_OUT,84
128,Watford,TAKE_ON,266


In [31]:
teams = []
events = []
scores = []
score_type = []
for event in eventdf.event_type.unique():
    if event == 'GENERIC:generic':
        continue
    maxs = eventdf[eventdf[0] == eventdf[eventdf.event_type == event].max()[0]]
    mins = eventdf[eventdf[0] == eventdf[eventdf.event_type == event].min()[0]]
    for index, entry in maxs.iterrows():
        teams.append(entry.team)
        events.append(event)
        scores.append(entry[0])
        score_type.append('max')
    for index, entry in mins.iterrows():
        teams.append(entry.team)
        events.append(event)
        scores.append(entry[0])
        score_type.append('min')
maxminstats = {'teams': teams, 'events': events, 'scores': scores, 'score_type': score_type}
df = pd.DataFrame.from_dict(maxminstats)

In [32]:
eventdf.columns = ['team', 'event', 'score']
eventdf.to_csv('gameweek1.csv', index = False)

In [33]:
df[df.events == 'CARD']

,teams,events,scores,score_type
0,Brighton & Hove Albion,CARD,6,max
1,Brighton & Hove Albion,CARD,6,max
2,Chelsea,CARD,6,max
3,Crystal Palace,CARD,6,max
4,Newcastle United,CARD,6,max
5,AFC Bournemouth,CARD,1,min
6,Burnley,CARD,1,min
7,Crystal Palace,CARD,1,min
8,Everton,CARD,1,min
9,Leicester City,CARD,1,min


In [34]:
cards = eventdf[eventdf['event'] == 'CARD']

In [35]:
cards.to_csv("cards.csv", index = False)

In [36]:
gameweek1.iloc[88]

event_id                            178494422
event_type                               PASS
result                               COMPLETE
success                                  True
period_id                                   1
timestamp                          225.825971
end_timestamp                            None
ball_state                               None
ball_owning_team                         None
team_id                                  1611
player_id                              105338
coordinates_x                            0.33
coordinates_y                            0.82
end_coordinates_x                        0.51
end_coordinates_y                        0.87
receiver_player_id                        NaN
is_counter_attack                       False
pass_type                         SIMPLE_PASS
set_piece_type                            NaN
body_part_type                            NaN
goalkeeper_action_type                    NaN
card_type                         

In [37]:
eventdf

,team,event,score
0,AFC Bournemouth,CARD,1
1,AFC Bournemouth,FOUL_COMMITTED,3
2,AFC Bournemouth,GENERIC:generic,2
3,AFC Bournemouth,PASS,639
4,AFC Bournemouth,RECOVERY,62
...,...,...,...
140,West Ham United,GENERIC:generic,2
141,West Ham United,PASS,409
142,West Ham United,RECOVERY,58
143,West Ham United,SHOT,7


In [38]:
week1plus = gameweek1.groupby(['team', 'event_type']).size().reset_index()

In [39]:
week1neg = gameweek1.groupby(['opponent', 'event_type']).size().reset_index()

In [40]:
week1plus[(week1plus.team == week1neg.opponent) & (week1plus.event_type == week1neg.event_type)]

,team,event_type,0
138,West Ham United,CARD,2
139,West Ham United,FOUL_COMMITTED,5
140,West Ham United,GENERIC:generic,2
141,West Ham United,PASS,409
142,West Ham United,RECOVERY,58
143,West Ham United,SHOT,7
144,West Ham United,TAKE_ON,244


In [41]:
week1neg[week1neg.opponent == 'AFC Bournemouth']

,opponent,event_type,0
0,AFC Bournemouth,BALL_OUT,71
1,AFC Bournemouth,CARD,3
2,AFC Bournemouth,FOUL_COMMITTED,12
3,AFC Bournemouth,GENERIC:generic,1
4,AFC Bournemouth,PASS,284
5,AFC Bournemouth,RECOVERY,46
6,AFC Bournemouth,SHOT,16
7,AFC Bournemouth,TAKE_ON,218


In [42]:
week1plus[week1plus.team == 'AFC Bournemouth']

,team,event_type,0
0,AFC Bournemouth,CARD,1
1,AFC Bournemouth,FOUL_COMMITTED,3
2,AFC Bournemouth,GENERIC:generic,2
3,AFC Bournemouth,PASS,639
4,AFC Bournemouth,RECOVERY,62
5,AFC Bournemouth,SHOT,8
6,AFC Bournemouth,TAKE_ON,218


In [43]:
shots = gameweek1[gameweek1.event_type == 'SHOT']

In [44]:
shotdf = shots.groupby('team').size().reset_index()
shotdf.columns = ['team', 'for']

In [45]:
shotdf['against'] = shots.groupby('opponent').size().reset_index()[0]

In [46]:
shotdf

,team,for,against
0,AFC Bournemouth,8,16
1,Arsenal,27,7
2,Brighton & Hove Albion,6,14
3,Burnley,10,16
4,Chelsea,16,10
5,Crystal Palace,14,8
6,Everton,8,10
7,Huddersfield Town,8,14
8,Leicester City,7,27
9,Liverpool,13,8


In [47]:
shotdf.to_csv('shotdf.csv', index = False)

In [48]:
games[0].loc[:, ['event_type', 'player_id']]

,event_type,player_id
0,PASS,7906
1,PASS,105338
2,PASS,70122
3,PASS,7936
4,PASS,7918
...,...,...
1641,RECOVERY,8301
1642,BALL_OUT,None
1643,PASS,None
1644,TAKE_ON,14703


In [49]:
players.columns

Index(['weight', 'firstName', 'middleName', 'lastName', 'currentTeamId',
       'birthDate', 'height', 'wyId', 'foot', 'shortName',
       'currentNationalTeamId', 'passportArea_name', 'passportArea_id',
       'passportArea_alpha3code', 'passportArea_alpha2code', 'role_code2',
       'role_code3', 'role_name', 'birthArea_name', 'birthArea_id',
       'birthArea_alpha3code', 'birthArea_alpha2code'],
      dtype='object')

In [50]:
for col in players.columns:
    print(col)

weight
firstName
middleName
lastName
currentTeamId
birthDate
height
wyId
foot
shortName
currentNationalTeamId
passportArea_name
passportArea_id
passportArea_alpha3code
passportArea_alpha2code
role_code2
role_code3
role_name
birthArea_name
birthArea_id
birthArea_alpha3code
birthArea_alpha2code


In [51]:
player_eventdf = gameweek1.groupby(['player', 'event_type']).size().reset_index()
player_eventdf[player_eventdf.event_type == 'PASS'].sort_values(by=0, ascending = False)

,player,event_type,0
363,Harry Arter,PASS,105
228,David Josu\u00e9 Jim\u00e9nez Silva,PASS,104
342,Granit Xhaka,PASS,104
994,Toby Alderweireld,PASS,101
52,Andrew Surman,PASS,101
...,...,...,...
254,Diafra Sakho,PASS,2
772,Oliver McBurnie,PASS,2
987,Theo Walcott,PASS,2
527,Jonathan Walters,PASS,1


In [52]:
player_eventdf[player_eventdf.event_type == 'SHOT'].sort_values(by=0, ascending = False).head(20)

,player,event_type,0
370,Harry Kane,SHOT,6
31,Alex Oxlade-Chamberlain,SHOT,6
155,Christian Dannemann Eriksen,SHOT,6
717,Mohamed Salah Ghaly,SHOT,5
478,Jay Rodriguez,SHOT,5
630,Manolo Gabbiadini,SHOT,5
647,Marcus Rashford,SHOT,5
575,Kevin De Bruyne,SHOT,4
325,Gabriel Fernando de Jesus,SHOT,4
344,Granit Xhaka,SHOT,4


In [53]:
player_eventdf[player_eventdf.event_type == 'FOUL_COMMITTED'].sort_values(by=0, ascending = False).head(20)

,player,event_type,0
768,Nordin Amrabat,FOUL_COMMITTED,6
534,Jordan Ayew,FOUL_COMMITTED,5
798,Paul Pogba,FOUL_COMMITTED,4
903,Sam Vokes,FOUL_COMMITTED,4
791,Patrick van Aanholt,FOUL_COMMITTED,4
283,Emre Can,FOUL_COMMITTED,3
1041,Wayne Routledge,FOUL_COMMITTED,3
165,Christopher Schindler,FOUL_COMMITTED,3
187,Dale Stephens,FOUL_COMMITTED,3
956,Stefano Okaka Chuka,FOUL_COMMITTED,3


In [54]:
player_eventdf[player_eventdf.event_type == 'PASS'].sort_values(by=0, ascending = False).head(20)

,player,event_type,0
363,Harry Arter,PASS,105
228,David Josu\u00e9 Jim\u00e9nez Silva,PASS,104
342,Granit Xhaka,PASS,104
994,Toby Alderweireld,PASS,101
52,Andrew Surman,PASS,101
304,Fernando Luiz Rosa,PASS,98
764,Nicol\u00e1s Hern\u00e1n Otamendi,PASS,97
96,Ben Davies,PASS,94
520,John Stones,PASS,93
287,Eric Dier,PASS,90


In [55]:
gameweek1.groupby(['event_type', 'success']).size()

event_type  success
PASS        False      1518
            True       7507
SHOT        False       216
            True         29
TAKE_ON     False      1793
            True       1776
dtype: int64

In [56]:
shots.groupby(['player', 'success']).size().reset_index()

,player,success,0
0,Aaron Cresswell,False,1
1,Aaron Ramsey,False,3
2,Aaron Ramsey,True,1
3,Abdoulaye Doucour\u00e9,True,1
4,Ahmed Hegazy,False,2
...,...,...,...
130,Wayne Rooney,True,1
131,Wilfried Zaha,False,3
132,Xherdan Shaqiri,False,1
133,\u00c1lvaro Borja Morata Mart\u00edn,False,2
